# PyTorch Test for ECG Data

In [1]:
import torch
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import save_image
# Import Numpy
import numpy as np
# Plot Import
import matplotlib.pyplot as plt
# For timer
import time
# Import OS
import os
# import local libs
import ecg_tools_lite as et

if not os.path.exists('./mlp_img'):
    os.mkdir('./mlp_img')

In [2]:
if torch.cuda.is_available():
    print(" I haz cuda")
else:
    print("Haz no cuda")

 I haz cuda


In [3]:
def to_img(x):
    x = x.view(x.size(0), 1, 28, 28)
    return x

num_epochs = 20
batch_size = 128
learning_rate = 1e-3

def min_max_normalization(tensor, min_value, max_value):
    min_tensor = tensor.min()
    tensor = (tensor - min_tensor)
    max_tensor = tensor.max()
    tensor = tensor / max_tensor
    tensor = tensor * (max_value - min_value) + min_value
    return tensor


def tensor_round(tensor):
    return torch.round(tensor)


img_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Lambda(lambda tensor:min_max_normalization(tensor, 0, 1)),
    transforms.Lambda(lambda tensor:tensor_round(tensor))
])

# dataset = MNIST('./data', transform=img_transform, download=True)
# dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


class autoencoder(nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(360, 360, 16),
            nn.ReLU(True),
        )
        self.decoder = nn.Sequential(
            nn.Linear(360, 360, 16),
            nn.ReLU(True),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x


model = autoencoder().cuda()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(
                model.parameters()
            ) # lr=learning_rate, weight_decay=1e-5)

In [8]:
ecg_clean = et.load_ecg_file('all_all_fs360_e24.npy')
ecg_noisy = et.load_ecg_file('all_all_fs360_e06.npy')

trainloader = DataLoader( ecg_noisy, shuffle=True, num_workers=2 )
# testloader = DataLoader()

In [9]:
dataiter = iter(trainloader)

In [17]:
a = dataiter.next()

In [18]:
a

tensor([[[ 0.0051],
         [ 0.0031],
         [-0.0012],
         [-0.0021],
         [-0.0041],
         [-0.0089],
         [-0.0065],
         [-0.0070],
         [-0.0079],
         [-0.0074],
         [-0.0108],
         [-0.0132],
         [-0.0137],
         [-0.0127],
         [-0.0151],
         [-0.0166],
         [-0.0204],
         [-0.0214],
         [-0.0233],
         [-0.0243],
         [-0.0243],
         [-0.0243],
         [-0.0248],
         [-0.0262],
         [-0.0277],
         [-0.0310],
         [-0.0334],
         [-0.0392],
         [-0.0445],
         [-0.0479],
         [-0.0498],
         [-0.0512],
         [-0.0527],
         [-0.0527],
         [-0.0512],
         [-0.0512],
         [-0.0527],
         [-0.0551],
         [-0.0560],
         [-0.0541],
         [-0.0546],
         [-0.0512],
         [-0.0503],
         [-0.0493],
         [-0.0483],
         [-0.0435],
         [-0.0431],
         [-0.0406],
         [-0.0397],
         [-0.0382],


In [ ]:
# Added elapsed timer
elapsed_start = time.time() #timer

for epoch in range(num_epochs):
    running_loss = 0.0
    epoch_start = time.time() # timer
    for data in dataloader:
        img, _ = data
        img = img.view(img.size(0), -1)
        noisy_img = add_noise(img)
        noisy_img = Variable(noisy_img).cuda()
        img = Variable(img).cuda()
        # ===================forward=====================
        output = model(noisy_img)
        loss = criterion(output, img)
        MSE_loss = nn.MSELoss()(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # ===================log========================
    print('epoch [{}/{}], loss:{:.4f}, MSE_loss:{:.4f}'
          .format(epoch + 1, num_epochs, loss.data.item(), MSE_loss.data.item()
          )
          )
    epoch_end = time.time()
    print("Epoch run time: %.2f" %(epoch_end-epoch_start))
    if epoch % 10 == 0:
        x = to_img(img.cpu().data)
        x_hat = to_img(output.cpu().data)
        x_noisy = to_img(noisy_img.cpu().data)
        weights = to_img(model.encoder[0].weight.cpu().data)
        save_image(x, './mlp_img/x_{}.png'.format(epoch))
        save_image(x_hat, './mlp_img/x_hat_{}.png'.format(epoch))
        save_image(x_noisy, './mlp_img/x_noisy_{}.png'.format(epoch))
        save_image(weights, './filters/epoch_{}.png'.format(epoch))

elapsed_end = time.time()
elapsed_total = elapsed_end-elapsed_start
print("Total elapsed time: %.2f, (in minutes %3d)" %(elapsed_total, (elapsed_total/60)))

torch.save(model.state_dict(), './sim_autoencoder.pth')